In [1]:
import gurobipy as gp
import pandas as pd
from code_map import data_handling, utils, new_meters, final_markets
import numpy as np
import calendar 
from datetime import datetime
import pytz
import openpyxl
import os
import pickle


In [2]:
#return L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list
L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list = data_handling.load_collections("./one_day_collections.pkl")

In [3]:
print(f"Amount of markets : {len(M)}")
print(f"Amount of meters : {len(L)}")
print(f"Amount of meters with direction up or both : {len(L_u)}")
print(f"Amount of meters with direction down or both : {len(L_d)}")
print(f"Amount of hours : {len(H)}")

Amount of markets : 62
Amount of meters : 2189
Amount of meters with direction up or both : 2036
Amount of meters with direction down or both : 2051
Amount of hours : 24


In [5]:
one_day_model, x, y, w, d  = utils.run_optimization_model(L, M, H, F, Ir_hlm, Ia_hlm, Va_hm, Vp_h_m, Vm_m, R_m, R_h_l, Fu_h_l, Fd_h_l, dominant_directions, compatible_list, "test_model.log", "one_day_model")

Set parameter LogFile to value "test_model.log"
Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[rosetta2] - Darwin 23.1.0 23B92)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6572952 rows, 3260208 columns and 19765965 nonzeros
Model fingerprint: 0x2d5014d8
Model has 133324 quadratic objective terms
Model has 1488 quadratic constraints
Variable types: 0 continuous, 3260208 integer (3260208 binary)
Coefficient statistics:
  Matrix range     [1e-06, 2e+03]
  QMatrix range    [1e-06, 8e-01]
  Objective range  [5e-06, 2e+05]
  QObjective range [1e-06, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [2e+00, 1e+02]
Found heuristic solution: objective -0.0000000
Presolve removed 6517389 rows and 1855807 columns (presolve time = 5s) ...
Presolve removed 6517402 rows and 1855807 columns
Presolve time: 7.57s
Presolved: 67386 rows, 1410134 columns, 268298 nonzeros
Variabl

In [21]:
type(x)

dict

In [22]:
fff = {}

In [15]:
def test_solution_validity(x, y, w, Va_hm, L, M, H, dominant_directions, F):
    """ function to test the validity of the solution provided by a solver

    Args:
        x (dict): dictionary of the binary variable which tells if an asset is connected to a market
        y (dict): dictionary of the binary variable which tells if a market has any bids
        w (dict): dictionary of the binary variable which tells if a market is activated
        Va_hm (dict): dictionary of the volume from the activation markets for each hour and market
        L (list(PowerMeter)): list of powermeter objects with the data for each meter within the timeframe
        M (list(ReserveMarket)): list of reservemarket objects with the data for each market within the timeframe
        H (list(pd.TimeStamp)): list of hourly timestamps within the timeframe
        dominant_directions (list(str)): list of the dominant direction for each hour
        F (pd.DataFrame): dictionary for frequency data
    Returns:
        str : a string that tells if the solution is valid. If not valid, the function will raise an error
    """
    for h, hour in enumerate(H):
        for l, load in enumerate(L):
            # Each asset can only be connected to one market at a time
            assert sum(x[h, l, m].X for m in range(len(M))) <= 1, f"Asset {l} connected to multiple markets at hour {h}"
            for m, market in enumerate(M):
                # Directionality constraints
                if load.direction == "up" and market.direction == "down":
                    assert x[h, l, m].X == 0, f"Up-direction asset {l} connected to down-direction market {m} at hour {h}"
                elif load.direction == "down" and market.direction == "up":
                    assert x[h, l, m].X == 0, f"Down-direction asset {l} connected to up-direction market {m} at hour {h}"
                #elif market.direction == "both" and load.direction != "both":
                    #assert x[h, l, m].X == 0, f"Asset {l} with specific direction connected to both-direction market {m} at hour {h}"
                elif market.area != load.area:
                    assert x[h, l, m].X == 0, f"Asset {l} in area {load.area} connected to market {m} in area {market.area} at hour {h}"
                
                # Response time constraints
                assert x[h, l, m].X * load.response_time <= market.response_time * y[h, m].X, f"Asset {l} connected to market {m} at hour {h} violates response time constraint"
                
        for m, market in enumerate(M):
            # Connect the binary variables by using big M
            assert sum(x[h, l, m].X for l in range(len(L))) <= len(L) * y[h, m].X, f"More than allowed assets connected at hour {h} to market {m}"

            #total_flex_volume = sum(x[h, l, m].X * load.flex_volume["value"].loc[load.flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L))

            # Min volume constraint
            if market.direction == "up":
                total_flex_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
            elif market.direction == "down":
                total_flex_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
            else: # direction = "both"
                if dominant_directions[h] == "up":
                    total_flex_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
                else:
                    total_flex_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
            
            assert round(total_flex_volume, 5) >= market.min_volume * y[h, m].X, f"Minimum volume constraint violated at hour {h} for market {m}"
            
            # Max volume constraint for both capacity and activation
            if market.direction == "up":
                total_max_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")

            elif market.direction == "down":
                total_max_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")

            else:
                """if dominant_directions[h] == "up":
                    total_max_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
                else:
                    total_max_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")"""
                total_up_max_volume = sum(x[h, l, m].X * load.up_flex_volume["value"].loc[load.up_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "down")
                total_down_max_volume = sum(x[h, l, m].X * load.down_flex_volume["value"].loc[load.down_flex_volume["Time(Local)"] == hour].values[0] for l, load in enumerate(L) if load.direction != "up")
                up_frac, down_frac = F[h,m]
                total_max_volume = (total_up_max_volume * up_frac + total_down_max_volume * down_frac)
            
             # Assert the constraints
            activation_constraint = round(total_max_volume, 5)  * w[h,m].X <= Va_hm[h,m]
            assert activation_constraint, f"Activation constraint violated for hour {h}, market {m}"
            market_max_volume = market.volume_data.loc[market.volume_data["Time(Local)"] == hour].values[0][1]
            assert total_max_volume <= market_max_volume * y[h,m].X, f"Maximum volume constraint violated at hour {h} for market {m}"
    return "Solution is valid"

In [16]:
test_solution_validity(x, y, w, Va_hm, L, M, H, dominant_directions= dominant_directions, F = F)

'Solution is valid'

In [17]:
new_x_values = {(h, l, m): one_day_model.getVarByName(f"x_{h}_{l}_{m}").X for h in range(len(H)) for l in range(len(L)) for m in range(len(M))}

In [20]:
# Load the saved values
with open('current_x_values.pkl', 'rb') as f:
    original_x_values = pickle.load(f)

old_dict = utils.get_market_count_dict(original_x_values, H, L, M, dominant_directions)
new_dict = utils.get_market_count_dict(new_x_values, H, L, M, dominant_directions)

differences = {}
for key in old_dict:
    if not old_dict[key].equals(new_dict[key]):
        differences[key] = (new_dict[key], old_dict[key])
        

for key, (orig_val, mod_val) in differences.items():
    print(f"Difference for hour {key}: \n Original={display(orig_val)}, \n  Modified={display(mod_val)}")


,Market,Hour,Asset Count,Total Flex Volume
0,FCR_N_D_1_NO1,2023-06-26 00:00:00+02:00,205,1.035558
1,aFRR down_NO5,2023-06-26 00:00:00+02:00,513,3.585945


,Market,Hour,Asset Count,Total Flex Volume
0,FCR_N_D_1_NO5,2023-06-26 00:00:00+02:00,119,2.259551
1,aFRR down_NO5,2023-06-26 00:00:00+02:00,547,0.272593
2,aFRR up_NO5,2023-06-26 00:00:00+02:00,200,1.000000


Difference for hour 2023-06-26 00:00:00+02:00: 
 Original=None, 
  Modified=None


,Market,Hour,Asset Count,Total Flex Volume
0,FCR_N_D_1_NO5,2023-06-26 02:00:00+02:00,257,2.288903


,Market,Hour,Asset Count,Total Flex Volume
0,FCR_N_D_1_NO5,2023-06-26 02:00:00+02:00,311,0.212157


...

,Market,Hour,Asset Count,Total Flex Volume
0,FCR_N_D_1_NO1,2023-06-26 22:00:00+02:00,240,0.418037
1,FCR_N_D_1_NO5,2023-06-26 22:00:00+02:00,257,0.189326
2,aFRR down_NO3,2023-06-26 22:00:00+02:00,114,0.155860
3,aFRR down_NO5,2023-06-26 22:00:00+02:00,295,0.324403
4,aFRR up_NO5,2023-06-26 22:00:00+02:00,332,1.775116


Difference for hour 2023-06-26 22:00:00+02:00: 
 Original=None, 
  Modified=None


,Market,Hour,Asset Count,Total Flex Volume
0,aFRR down_NO5,2023-06-26 23:00:00+02:00,541,1.912328


,Market,Hour,Asset Count,Total Flex Volume
0,FCR_N_D_1_NO1,2023-06-26 23:00:00+02:00,256,0.308153
1,FCR_N_D_1_NO5,2023-06-26 23:00:00+02:00,263,0.245672
2,aFRR down_NO5,2023-06-26 23:00:00+02:00,302,0.213090
3,aFRR up_NO5,2023-06-26 23:00:00+02:00,315,1.382461


Difference for hour 2023-06-26 23:00:00+02:00: 
 Original=None, 
  Modified=None


In [ ]:

# Save these values
with open('current_x_values_for_week.pkl', 'wb') as f:
    pickle.dump(new_x_values, f)
                